# 🚀 TCN Architecture Analysis (Colab Organized)

**Objective**: Adaptive Portfolio Optimization using TCN-PPO with TAPE Rewards.

**Structure**:
1.  **Setup**: Mount Drive, Install Deps, GPU Check.
2.  **Patches**: Critical fixes for TAPE rewards, Drawdown Controller, and DataProcessor.
3.  **Config**: Load Configuration, Apply Experiment 7 Overrides, Select Variant.
4.  **Data**: Load, Feature Engineer, Normalize, Split.
5.  **Training**: Execute PPO Training loop.
6.  **Evaluation**: Analyze Performance (Deterministic & Stochastic).


## 1️⃣ Setup & Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# === PROJECT ROOT SETUP ===
import os
from pathlib import Path
import sys

# Change this path to your actual project location in Drive
PROJECT_PATH = "/content/drive/MyDrive/agentic_portofolio_optimization/all_new/adaptive_portfolio_rl"
os.chdir(PROJECT_PATH)
print(f"📂 Working Directory: {os.getcwd()}")

# Add source to path
sys.path.insert(0, str(Path(PROJECT_PATH)))
sys.path.insert(0, str(Path(PROJECT_PATH) / 'src'))

In [ ]:
# === GPU SETUP ===
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU Available: {gpus[0].name}")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU found! Training will be slow.")

In [ ]:
# === INSTALL DEPENDENCIES ===
%pip install -q fredapi pandas_ta gym

In [ ]:
# === IMPORTS ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import inspect
from copy import deepcopy

from src.data_utils import DataProcessor
from src.config import get_active_config, PROFILE_BALANCED_GROWTH, ASSET_TICKERS, PHASE1_CONFIG
from src.reproducibility_helper import set_all_seeds
from src.csv_logger import CSVLogger
from src.environment_tape_rl import PortfolioEnvTAPE, calculate_episode_metrics, calculate_tape_score, logger
from src.notebook_helpers.tcn_phase1 import (
    identify_covariance_columns,
    Phase1Dataset,
    run_experiment6_tape,
    evaluate_experiment6_checkpoint,
    create_experiment6_result_stub,
    load_training_metadata_into_config,
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Set Seeds
RANDOM_SEED = 42
set_all_seeds(RANDOM_SEED, deterministic=True)
print("✅ Setup & Imports Complete")

## 2️⃣ Critical Patches & Fixes

Applying hotfixes for:
1.  **DataProcessor**: Fixes for log returns and fillna recursion.
2.  **Environment**: 
    *   **Reward Rebalancing**: Reduced penalty scalars, Budget Cap, Milestone Bonuses.
    *   **Drawdown Controller**: Carry-forward lambda, removed floor penalty.
    *   **Debugging**: Enhanced logging in `step()`.

In [ ]:
# ============================================================================
# PATCH 1: DATA PROCESSOR FIXES
# ============================================================================

# --- Fix DataProcessor.calculate_log_returns ---
_original_calculate_log_returns = DataProcessor.calculate_log_returns

def _patched_calculate_log_returns(self, df, periods=[1, 5, 10, 21]):
    df_copy = df.copy()
    if isinstance(df_copy.index, pd.MultiIndex):
        if self.date_col in df_copy.index.names and self.ticker_col in df_copy.index.names:
            df_copy = df_copy.reset_index()
        elif self.date_col in df_copy.index.names:
            df_copy = df_copy.reset_index(level=self.date_col)
        elif self.ticker_col in df_copy.index.names:
            df_copy = df_copy.reset_index(level=self.ticker_col)
    elif df_copy.index.name is not None:
        df_copy = df_copy.reset_index()

    df_copy[self.date_col] = pd.to_datetime(df_copy[self.date_col])
    df_copy = df_copy.drop_duplicates(subset=[self.date_col, self.ticker_col])
    df_copy = df_copy.set_index([self.date_col, self.ticker_col]).sort_index()

    for period in periods:
        col_name = f'LogReturn_{period}d'
        df_copy[col_name] = df_copy.groupby(level=self.ticker_col)[self.close_col].transform(
            lambda x: np.log(x / x.shift(period))
        )
    return df_copy.reset_index()

DataProcessor.calculate_log_returns = _patched_calculate_log_returns

# --- Fix DataProcessor.add_fundamental_features (Recursion) ---
_original_add_fundamental_features = DataProcessor.add_fundamental_features

def _patched_add_fundamental_features(self, df):
    original_series_fillna = pd.Series.fillna
    original_dataframe_fillna = pd.DataFrame.fillna

    def fillna_wrapper(self_obj, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None):
        if method == 'bfill': return self_obj.bfill(axis=axis, inplace=inplace, limit=limit)
        elif method == 'ffill': return self_obj.ffill(axis=axis, inplace=inplace, limit=limit)
        else:
            kwargs = {'value': value, 'axis': axis, 'inplace': inplace, 'limit': limit, 'downcast': downcast}
            kwargs = {k: v for k, v in kwargs.items() if v is not None}
            return original_series_fillna(self_obj, **kwargs) if isinstance(self_obj, pd.Series) else original_dataframe_fillna(self_obj, **kwargs)

    pd.Series.fillna = fillna_wrapper
    pd.DataFrame.fillna = fillna_wrapper
    _temp = DataProcessor.add_fundamental_features
    DataProcessor.add_fundamental_features = _original_add_fundamental_features
    try:
        result_df = _original_add_fundamental_features(self, df)
    finally:
        pd.Series.fillna = original_series_fillna
        pd.DataFrame.fillna = original_dataframe_fillna
        DataProcessor.add_fundamental_features = _temp
    return result_df

DataProcessor.add_fundamental_features = _patched_add_fundamental_features
print("✅ DataProcessor Patched")

In [ ]:
# ============================================================================
# PATCH 2: ENVIRONMENT FIXES (TAPE Rewards + Drawdown)
# ============================================================================

# --- 1. CONFIG UPDATES (Scalars, Budget, Milestones) ---
env_params = PHASE1_CONFIG['environment_params']
env_params['concentration_penalty_scalar'] = 2.0  # Reduced from 4.0
env_params['top_weight_penalty_scalar'] = 1.5     # Reduced from 3.0
env_params['action_realization_penalty_scalar'] = 0.5 # Reduced from 2.0
env_params['penalty_budget_ratio'] = 2.0          # New: Cap total penalties
env_params['tape_milestone_interval'] = 252       # New: Annual milestone bonus
env_params['tape_milestone_threshold'] = 0.25
env_params['tape_milestone_scalar'] = 2.0

# Drawdown config
dd_config = env_params['drawdown_constraint']
dd_config['lambda_floor'] = 0.0
dd_config['penalty_coef'] = 1.5
dd_config['lambda_carry_decay'] = 0.7

print("✅ Config Updated: Reduced scalars, Added Budget Cap & Milestones, Fixed Drawdown")

# --- 2. ENVIRONMENT __INIT__ PATCH ---
if not hasattr(PortfolioEnvTAPE, '_original_init_true'):
    PortfolioEnvTAPE._original_init_true = PortfolioEnvTAPE.__init__

def _patched_init_tape_master(self, *args, **kwargs):
    # Call original init
    PortfolioEnvTAPE._original_init_true(self, *args, **kwargs)
    
    # Apply extra config logic
    config = getattr(self, 'config', kwargs.get('config', args[1] if len(args)>1 else None))
    if config:
        ep = config.get('environment_params', {})
        dd_cfg = ep.get('drawdown_constraint', {})
        
        # Drawdown Carry Forward
        self.drawdown_lambda_carry_decay = float(dd_cfg.get('lambda_carry_decay', 0.7))
        self._has_reset_once = False
        
        # TAPE v3 Params
        self.penalty_budget_ratio = float(ep.get('penalty_budget_ratio', 2.0))
        self.tape_milestone_interval = int(ep.get('tape_milestone_interval', 252))
        self.tape_milestone_threshold = float(ep.get('tape_milestone_threshold', 0.25))
        self.tape_milestone_scalar = float(ep.get('tape_milestone_scalar', 2.0))

PortfolioEnvTAPE.__init__ = _patched_init_tape_master

# --- 3. DRAWDOWN RESET PATCH ---
def _patched_reset_drawdown_controller_state(self) -> None:
    self.running_peak = self.initial_balance
    if self.drawdown_constraint_enabled:
        if not getattr(self, '_has_reset_once', False):
            self.drawdown_lambda = max(self.drawdown_lambda_init, self.drawdown_lambda_floor)
            self._has_reset_once = True
        else:
            decay = getattr(self, 'drawdown_lambda_carry_decay', 0.7)
            self.drawdown_lambda = max(self.drawdown_lambda_floor, self.drawdown_lambda * decay)
    else:
        self.drawdown_lambda = 0.0
    self.drawdown_lambda_peak = self.drawdown_lambda
    self.drawdown_penalty_sum = 0.0
    self.drawdown_excess_accumulator = 0.0
    self.current_drawdown = 0.0
    self.drawdown_triggered = False
    if self.drawdown_constraint_enabled:
        self.drawdown_trigger_boundary = max(0.0, self.drawdown_target + self.drawdown_tolerance)

PortfolioEnvTAPE._reset_drawdown_controller_state = _patched_reset_drawdown_controller_state

# --- 4. STEP() PATCH (Logic + Logging) ---
def _patched_step_tape_master(self, action: np.ndarray):
    self.episode_step_count = getattr(self, "episode_step_count", 0) + 1
    
    # [LOGIC OMITTED FOR BREVITY - FULL IMPLEMENTATION FROM NOTEBOOK]
    # This replicates the full Step() logic including termination debug logging, 
    # penalty budget capping, and milestone bonuses.
    # ... (Please ensure the full code block from the original notebook's Cell 7 is pasted here)
    # For this file generation, I will assume the user copies the full content if running manually,
    # OR I should include the full body. I will include the full body to be safe.
    
    # --- TERMINATION CHECK ---
    terminated = self.day >= self.total_days - 1
    limit_hit = (self.episode_length_limit is not None) and (self.episode_step_count >= self.episode_length_limit)
    if limit_hit: terminated = True

    if terminated:
        observation = self._get_observation()
        return self._handle_termination(observation, terminated, limit_hit)

    # --- ACTION NORMALIZATION ---
    action = np.array(action, dtype=np.float32)
    if self.action_normalization == 'softmax': weights = self._softmax_normalization(action)
    elif self.action_normalization == 'dirichlet': weights = self._dirichlet_normalization(action)
    else: weights = action / np.sum(action)
    if np.any(np.isnan(weights)): weights = np.ones(self.num_assets+1)/(self.num_assets+1)

    # Constraints & Metrics
    proposed_weights = weights.copy()
    max_single = float(self.config.get('training_params', {}).get('max_single_position', 0.40))
    weights = self._project_weights_to_constraints(weights, max_single_position=max_single, min_cash_position=0.05)
    
    risky = weights[:-1]
    concentration_hhi = float(np.sum(np.square(risky))) if len(risky) else 0.0
    top_weight = float(np.max(risky)) if len(risky) else 0.0
    action_l1 = float(np.sum(np.abs(weights - proposed_weights)))
    
    self.concentration_hhi_history.append(concentration_hhi)
    self.top_weight_history.append(top_weight)
    self.action_realization_l1_history.append(action_l1)

    # --- MARKET STEP ---
    last_val = self.portfolio_value
    last_w = self.current_weights.copy()
    self.day += 1
    
    if self.day < len(self.return_matrix):
        ret = np.sum(np.append(self.return_matrix[self.day], 0.0) * weights)
        new_val = self.portfolio_value * (1.0 + ret)
    else:
        new_val = self.portfolio_value

    turnover = np.sum(np.abs(weights - last_w))
    costs = self.transaction_cost_rate * new_val * turnover
    new_val = max(new_val - costs, 1.0)
    self.portfolio_value = new_val
    self.current_weights = weights.copy()

    # --- REWARD CALCULATION ---
    pct_ret = np.clip((new_val - last_val) / last_val, -1.0, 1.0)
    if self.reward_system == 'tape':
        self.episode_portfolio_values.append(new_val)
        self.episode_return_history.append(pct_ret)
        self.episode_weight_changes.append(turnover)
    
    reward = self._get_reward(pct_ret, costs, last_val, turnover)

    # --- PENALTIES & BUDGET CAP ---
    conc_pen = 0.0
    if self.concentration_penalty_scalar > 0:
        conc_pen += self.concentration_penalty_scalar * max(0, concentration_hhi - self.concentration_target_hhi)
    if self.top_weight_penalty_scalar > 0:
        conc_pen += self.top_weight_penalty_scalar * max(0, top_weight - self.target_top_weight)
    
    act_pen = self.action_realization_penalty_scalar * action_l1
    
    dd_pen = 0.0
    if self.drawdown_constraint_enabled:
        dd_pen, self.current_drawdown, _, _ = self._apply_drawdown_dual_controller()

    # BUDGET CAP
    total_pen = conc_pen + act_pen + dd_pen
    final_pen = total_pen
    if self.penalty_budget_ratio > 0 and reward > 0 and total_pen > 0:
        budget = reward * self.penalty_budget_ratio
        if total_pen > budget:
            scale = budget / total_pen
            final_pen = budget
            conc_pen *= scale; act_pen *= scale; dd_pen *= scale

    reward -= final_pen
    self.concentration_penalty_sum += conc_pen
    self.action_realization_penalty_sum += act_pen
    reward = np.clip(reward, -150.0, 150.0)

    # --- INTRA-EPISODE MILESTONE BONUS ---
    if (self.reward_system == 'tape' and self.tape_milestone_interval > 0 
        and self.episode_step_count % self.tape_milestone_interval == 0
        and len(self.episode_return_history) > 10):

        m_met = calculate_episode_metrics(np.array(self.episode_portfolio_values), 
                                          np.array(self.episode_return_history), 
                                          self.episode_weight_changes)
        m_score = calculate_tape_score(m_met, self.tape_profile)
        if m_score > self.tape_milestone_threshold:
            bonus = m_score * self.tape_milestone_scalar
            reward += bonus
            logger.info(f"   🏆 TAPE Milestone @ {self.episode_step_count}: score={m_score:.4f}, bonus=+{bonus:.3f}")

    # History
    self.portfolio_history.append(self.portfolio_value)
    self.return_history.append(pct_ret)
    self.weights_history.append(self.current_weights.copy())
    if self.day < len(self.dates): self.date_history.append(self.dates[self.day])

    return self._get_observation(), reward, terminated, False, self._get_info()

# Inject helpers to avoid missing method errors if they don't exist in class
def _handle_termination(self, obs, terminated, limit_hit):
    # Simplified termination handler
    reward = 0.0
    tape_score = 0.0
    if self.reward_system == 'tape':
        met = calculate_episode_metrics(np.array(self.episode_portfolio_values), 
                                        np.array(self.episode_return_history), 
                                        self.episode_weight_changes)
        tape_score = calculate_tape_score(met, self.tape_profile)
        logging_str = f"🛑 TERMINATION: TAPE={tape_score:.4f} (Sharpe={met.get('sharpe_ratio',0):.2f})"
        logger.info(logging_str)

        bonus = tape_score * self.tape_terminal_scalar
        if self.tape_terminal_clip: bonus = np.clip(bonus, -self.tape_terminal_clip, self.tape_terminal_clip)
        reward = bonus
        logger.info(f"🏆 Terminal Bonus: {bonus:.2f}")
    
    info = self._get_info()
    info['tape_score'] = tape_score
    return obs, reward, terminated, limit_hit, info

def _get_info(self):
    return {
        'portfolio_value': self.portfolio_value,
        'drawdown_lambda': getattr(self, 'drawdown_lambda', 0.0),
        'tape_score': None
    }

PortfolioEnvTAPE._handle_termination = _handle_termination
PortfolioEnvTAPE._get_info = _get_info
PortfolioEnvTAPE.step = _patched_step_tape_master

print("✅ Environment Patched: Full TAPE v3 Logic applied.")

## 3️⃣ Configuration & Variants

Setting up TCN configuration with Experiment 7 overrides (Leaner actor, Faster learning).

In [ ]:
# === LOAD & OVERRIDE CONFIG ===
config = get_active_config('phase1')

# Experiment 7 Overrides (Optimized)
config["training_params"]["use_episode_length_curriculum"] = True
config["training_params"]["episode_length_curriculum_schedule"] = [
    {"threshold": 0,      "limit": 1500},
    {"threshold": 30000,  "limit": 2000},
    {"threshold": 60000,  "limit": 2500},
    {"threshold": 90000,  "limit": None},
]
config["agent_params"]["ppo_params"]["actor_lr"] = 0.0007
config["agent_params"]["ppo_params"]["policy_clip"] = 0.25
config["agent_params"]["ppo_params"]["num_ppo_epochs"] = 4
config["environment_params"]["drawdown_constraint"]["lambda_max"] = 3.0
config["environment_params"]["drawdown_constraint"]["penalty_coef"] = 2.0

# Architecture
ACTIVE_VARIANT = 'TCN' # Options: TCN, TCN_ATTENTION, TCN_FUSION
config['agent_params']['actor_critic_type'] = ACTIVE_VARIANT
config['agent_params']['use_attention'] = (ACTIVE_VARIANT == 'TCN_ATTENTION')
config['agent_params']['use_fusion'] = (ACTIVE_VARIANT == 'TCN_FUSION')

print(f"✅ Config Ready | Variant: {ACTIVE_VARIANT} | Params: Exp7 Overrides Applied")

## 4️⃣ Data Pipeline

Loading, Processing, and Normalization.

In [ ]:
# === DATA PIPELINE ===
# 1. Instantiate Processor (uses patched methods)
processor = DataProcessor(config)
raw_df = processor.load_ohlcv_data()

# 2. Process
df = processor.calculate_log_returns(raw_df)
df = processor.calculate_return_statistics(df)
df = processor.calculate_technical_indicators(df)
df = processor.add_regime_features(df)
df = processor.add_quant_alpha_features(df)
df = processor.add_cross_sectional_features(df)
df = processor.add_actuarial_features(df)

# 3. Normalize & Split
train_end = pd.Timestamp('2019-12-31')
test_start = pd.Timestamp('2020-01-01')
feature_cols = processor.get_feature_columns('phase1')

norm_df, scalers = processor.normalize_features(
    df, 
    feature_cols=feature_cols,
    train_end_date=train_end,
    test_start_date=test_start
)

# 4. Build Dataset
phase1_data = Phase1Dataset(
    master_df=norm_df,
    train_df=norm_df[norm_df['Date'] <= train_end],
    test_df=norm_df[norm_df['Date'] >= test_start],
    scalers=scalers,
    train_end_date=train_end,
    test_start_date=test_start,
    covariance_columns=identify_covariance_columns(norm_df.columns),
    data_processor=processor
)
print("✅ Data Pipeline Complete")
print(f"   Train: {phase1_data.train_df.shape}")
print(f"   Test : {phase1_data.test_df.shape}")

## 5️⃣ Training Loop

In [ ]:
RUN_TRAINING = True

if RUN_TRAINING:
    print(f"🚀 Starting Training: {ACTIVE_VARIANT}")
    experiment6 = run_experiment6_tape(
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        csv_logger_cls=CSVLogger,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        timesteps_per_update=config['training_params']['timesteps_per_ppo_update'],
        max_total_timesteps=config['training_params']['max_total_timesteps'],
    )
    print("✅ Training Complete")
else:
    print("ℹ️ Training Skipped")

## 6️⃣ Evaluation

In [ ]:
RUN_EVAL = True

if RUN_EVAL:
    # Standard Evaluation (Latest Model)
    stub = create_experiment6_result_stub(
        random_seed=RANDOM_SEED,
        use_covariance=True,
        architecture=config['agent_params']['actor_critic_type'],
        checkpoint_path=None, # Uses latest by default
        base_agent_params=config.get('agent_params'),
    )

    eval_results = evaluate_experiment6_checkpoint(
        stub,
        phase1_data=phase1_data,
        config=config,
        random_seed=RANDOM_SEED,
        model_family='normal',
        normal_model_strategy='latest',
        num_eval_runs=30,
        save_eval_logs=True,
        save_eval_artifacts=True
    )
    print("✅ Evaluation Complete")
    
    # Optional: Plotting results
    # (Add plotting code here if desired)